# Giskard Robot playground

Giskard version: [mujoco_actions_devel](https://github.com/SemRoCo/giskardpy/tree/37b4048c3787c1f5f0370ca4e8ee10754eaa12c9)

## 1.Parameters and helper functions

In [ ]:
import os
import psutil
import subprocess
import rospy
from ipywidgets import Button, Layout, GridBox, Box, FloatSlider
from IPython.display import display, Markdown, IFrame
from sidecar import Sidecar

# Parameters and Helper function
# Directory of the ROS launch files
LAUNCH_FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))
# To display rvizweb on the left
SIDECAR = None
# To manage the roslaunch process in the background
LAUNCH_PROCESS = None
SELECTED_ROBOT = 'hsr_mujoco'
CMD_VEL_TOPIC = '/cmd_vel'

try:
    RIVZWEB = IFrame(src=rospy.get_param('/rvizweb/jupyter_proxy_url'), width='100%', height='100%')
except Exception as e:
    RIVZWEB = None

# Fetch parameter from the URL
try:
    SELECTED_ROBOT = rospy.get_param('/nbparam_robot')
    print(f"Selected Robot: {SELECTED_ROBOT}")
except Exception as e:
    print(f"Will launch the default robot: {SELECTED_ROBOT}")

# Check if the docker image runs on a PC with display, if true, will launch rviz and mujoco GUI window
try:
    HAS_DISPLAY = 'true' if os.environ['DISPLAY'] != ':100' else 'false'
except KeyError:
    HAS_DISPLAY = 'false'

# Popup the rvizweb on the left panel
def open_rvizweb():
    global SIDECAR
    if SIDECAR is not None:
        SIDECAR.close()
    if RIVZWEB is not None:
        SIDECAR = Sidecar(title='RVIZWEB', anchor='right')
        with SIDECAR:
            display(RIVZWEB)

# Execute the roslaunch command (will restart roscore, same as restart the notebook kernel)
def launch_robot(config):
    global LAUNCH_PROCESS
    global CMD_VEL_TOPIC
    if LAUNCH_PROCESS is not None:
        LAUNCH_PROCESS.terminate()
        LAUNCH_PROCESS.wait()
    if 'cmd_vel' in config and config['cmd_vel'] is not None: 
        CMD_VEL_TOPIC = config['cmd_vel']
    launchfile = os.path.join(LAUNCH_FILE_DIR, f"{config['launchfile']}.launch")
    command = [
        'roslaunch',
        launchfile,
        f"gui:={HAS_DISPLAY}",
        'mujoco_suffix:=' + ('' if HAS_DISPLAY == 'true' else '_headless')
    ]
    LAUNCH_PROCESS = psutil.Popen(command,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL)
    
    open_rvizweb()
    print(f"""
        Executing command: 
        {' '.join(command)}
        in the background, the output will be hidden. 
        To check the output please execute it in a Terminial!
    """)

# Create button widgets
def create_button(config):
    btn = Button(
        description=config['name'],
        layout=Layout(width='auto', height='40px'),
        style={'font_size':'1rem'},
        tooltip=f"Launch robot: {config['name']}"
    )
    btn.on_click(lambda b: launch_robot(config))
    return btn

## 2. Launch Robot and envrionment

In [ ]:
ROBOT_LIST = [
    {
        'name': 'PR2 (Mujoco)',
        'id': 'pr2_mujoco',
        'launchfile': 'pr2_mujoco',
        'cmd_vel': '/pr2/cmd_vel'
    },
    {
        'name': 'HSR (Mujoco)',
        'id': 'hsr_mujoco',
        'launchfile': 'hsr_mujoco',
        'cmd_vel': '/hsrb4s/cmd_vel'
    },
    {
        'name': 'PR2 (PyBullet)',
        'id': 'pr2',
        'launchfile': 'pr2_standalone'
    },
    {
        'name': 'HSR (PyBullet)',
        'id': 'hsr',
        'launchfile': 'hsr_standalone'
    },
    {
        'name': 'Tiago',
        'id': 'tiago',
        'launchfile': 'tiago_standalone'
    },    
    {
        'name': 'Stretch',
        'id': 'stretch',
        'launchfile': 'stretch_standalone'
    },
    {
        'name': 'Donbot',
        'id': 'donbot',
        'launchfile': 'donbot_standalone'
    },
    # {
    #     'name': 'Tracy',
    #     'id': 'tracy',
    #     'launchfile': 'tracy_standalone'
    # },
    # {
    #     'name': 'Boxy',
    #     'id': 'boxy',
    #     'launchfile': 'boxy_standalone'
    # },
    # {
    #     'name': 'Armar',
    #     'id': 'armar',
    #     'launchfile': 'armar_standalone'
    # }
]


# Launch the robot pass from the url parameter or the default robot
try:
    robot_config = next(i for i in ROBOT_LIST if i['id'] == SELECTED_ROBOT)
    launch_robot(robot_config)
    display(Markdown(f"### Robot **'{robot_config['name']}'** Launched"))
except Exception as e:
    print(e)

# Display Launch widgets
display(Markdown('#### Click the buttons below to launch a new robot.'))
display(GridBox([create_button(robot) for robot in ROBOT_LIST],
                layout=Layout(grid_template_columns="repeat(2, 50%)")))

#### Control Robot base with ROS topic "cmd_vel" (Mujoco simulator only)

In [ ]:
from geometry_msgs.msg import Twist, Vector3
import rospy

rospy.init_node('notebook_playground')
cmd_vel_pub = rospy.Publisher(CMD_VEL_TOPIC, Twist, queue_size=5)
cmd_vel_msg = Twist()

In [ ]:
# Move 1 meter forward in one second
cmd_vel_msg.linear.x = 1
cmd_vel_pub.publish(cmd_vel_msg)
rospy.sleep(1)
cmd_vel_msg.linear.x = 0
cmd_vel_pub.publish(cmd_vel_msg)

In [ ]:
# slider for moving velocity
linear_x = FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='Moving',
    orientation='vertical',
    readout=True,
    readout_format='.1f',
)

def on_linear_x_change(v):
    cmd_vel_msg.linear.x = v
    cmd_vel_pub.publish(cmd_vel_msg)

linear_x.observe(lambda v: on_linear_x_change(v['new']), names='value')

# slider for rotation velocity
angular_z = FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='Rotation',
    readout=True,
    readout_format='.1f',
)

def on_angular_z_change(v):
    cmd_vel_msg.angular.z = -v
    cmd_vel_pub.publish(cmd_vel_msg)

angular_z.observe(lambda v: on_angular_z_change(v['new']), names='value')

display(Markdown('#### Robot Base Velocity Controller:'))
display(Box([linear_x, angular_z]))

In [ ]:
# Control robot with joystick
from ipywidgets import Controller
joystick = Controller()
max_speed = 0.6
max_rotate = 0.6
axes = [0 ,0, 0, 0]
joystick_thredshold = 0.2

def joystick_callback(v):
    global axes
    new_axes = [(round(i.value, ndigits=1) if abs(i.value) > joystick_thredshold else 0) for i in v.owner.axes]
    if (new_axes[0] - axes[0]) != 0 or (new_axes[1] - axes[1]) != 0:
        axes = new_axes
        cmd_vel_msg.linear.x = -axes[1] * max_speed
        cmd_vel_msg.angular.z = -axes[0]  * max_rotate
        cmd_vel_pub.publish(cmd_vel_msg)

joystick.observe(joystick_callback, names="timestamp")
joystick

## 3. Control the robot with the Giskard Python Interface
### 3.1 Simple example
Reference: https://github.com/SemRoCo/giskardpy/wiki/Python-Interface

In [ ]:
import rospy
import random
from giskardpy.python_interface.python_interface import GiskardWrapper
from giskardpy.goals.joint_goals import JointPositionList
from giskardpy.monitors.joint_monitors import JointGoalReached
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, QuaternionStamped

rospy.init_node('notebook_playground')

def move_robot(gk_wrapper, pos, root_link='map', tip_link='base_link'):
    pos_stamp = PointStamped()
    pos_stamp.header.frame_id = root_link
    pos_stamp.point = pos
    gk_wrapper.add_default_end_motion_conditions()
    gk_wrapper.motion_goals.add_cartesian_position(
        root_link=root_link,
        tip_link=tip_link,
        goal_point=pos_stamp,
    )
    gk_wrapper.execute()

def rotate_robot(gk_wrapper, ori, root_link='map', tip_link='base_link'):
    ori_stamp = QuaternionStamped()
    ori_stamp.header.frame_id = root_link
    ori_stamp.quaternion = ori
    gk_wrapper.add_default_end_motion_conditions()
    gk_wrapper.motion_goals.add_cartesian_orientation(
        root_link=root_link,
        tip_link=tip_link,
        goal_orientation=ori_stamp,
    )
    gk_wrapper.execute()

def control_joint(gk_wrapper, joint_goal):
    gk_wrapper.motion_goals.add_joint_position(goal_state=joint_goal)
    gk_wrapper.add_default_end_motion_conditions()
    gk_wrapper.execute()

def list_links(gk_wrapper):
    return gk_wrapper.world.get_group_info(gk_wrapper.world.get_group_names()[0]).links

def list_joints(gk_wrapper):
    return gk_wrapper.world.get_controlled_joints(gk_wrapper.world.get_group_names()[0])


#### 3.1.0 Get Giskard python API wrapper

In [ ]:
giskard_wrapper = GiskardWrapper()

#### 3.1.1 Move robot around

In [ ]:
# List all the links
print(list_links(giskard_wrapper))

# Move the robot to a absolute position
move_robot(giskard_wrapper, Point(1.2, 0, 0))

# Rotate robot body (absolute Coordinate)
# rotate_robot(giskard_wrapper, Quaternion(0, 0, -0.1, 1))

#### 3.1.2 Control joints

In [ ]:
# Control joints
print(list_joints(giskard_wrapper))
control_joint(giskard_wrapper, {
    'wrist_roll_joint': -2,
    'torso_lift_joint': 0.2
})

### 3.2 Complex example
>Note: PR2 only !

In [ ]:
import rospy
from geometry_msgs.msg import PoseStamped

from giskardpy.goals.joint_goals import JointPositionList
from giskardpy.monitors.joint_monitors import JointGoalReached
from giskardpy.python_interface.python_interface import GiskardWrapper

rospy.init_node('notebook_playground')

# %% Define goals for later
right_arm_goal = {'r_shoulder_pan_joint': -1.7125,
                  'r_shoulder_lift_joint': -0.25672,
                  'r_upper_arm_roll_joint': -1.46335,
                  'r_elbow_flex_joint': -2.12,
                  'r_forearm_roll_joint': 1.76632,
                  'r_wrist_flex_joint': -0.10001,
                  'r_wrist_roll_joint': 0.05106}

left_arm_goal = {'l_shoulder_pan_joint': 1.9652,
                 'l_shoulder_lift_joint': - 0.26499,
                 'l_upper_arm_roll_joint': 1.3837,
                 'l_elbow_flex_joint': -2.12,
                 'l_forearm_roll_joint': 16.99,
                 'l_wrist_flex_joint': - 0.10001,
                 'l_wrist_roll_joint': 0}

base_goal = PoseStamped()
base_goal.header.frame_id = 'map'
base_goal.pose.position.x = 2
base_goal.pose.orientation.w = 1

# %% init ros node and Giskard Wrapper.
# This assumes that roslaunch giskardpy giskardpy_pr2_standalone.launch is running.

rospy.loginfo('Instantiating Giskard wrapper.')
giskard_wrapper = GiskardWrapper()

# %% Remove everything but the robot.
# All world related operations are grouped under giskard_wrapper.world.
giskard_wrapper.world.clear()

# %% Monitors observe something and turn to True, if the condition is met. They don't cause any motions.
# All monitor related operations are grouped under giskard_wrapper.monitors.
# Let's define a few
# This one turns True when the length of the current trajectory % mod = 0
alternator = giskard_wrapper.monitors.add_alternator(mod=2)
# This one sleeps and then turns True
sleep1 = giskard_wrapper.monitors.add_sleep(1, name='sleep1')
# This prints a message and then turns True.
# With start_condition you can define which monitors need to be True in order for this one to become active
print1 = giskard_wrapper.monitors.add_print(message=f'{sleep1} done', start_condition=sleep1)
# You can also write logical expressions using "and", "or" and "not" to combine multiple monitors
sleep2 = giskard_wrapper.monitors.add_sleep(1.5, name='sleep2', start_condition=f'{print1} or not {sleep1}')

# %% Now Let's define some motion goals.
# We want to reach two joint goals, so we first define monitors for checking that end condition.
right_monitor = giskard_wrapper.monitors.add_joint_position(goal_state=right_arm_goal,
                                                            name='right pose reached',
                                                            start_condition=sleep1)
# You can use add_motion_goal to add any monitor implemented in giskardpy.monitor.
# All remaining parameters are forwarded to the __init__ function of that class.
# All specialized add_ functions are just wrappers for add_monitor.
left_monitor = giskard_wrapper.monitors.add_monitor(monitor_class=JointGoalReached.__name__,
                                                    goal_state=left_arm_goal,
                                                    name='left pose reached',
                                                    start_condition=sleep1,
                                                    threshold=0.01)

# We set two separate motion goals for the joints of the left and right arm.
# All motion goal related operations are groups under giskard_wrapper.motion_goals.
# The one for the right arm starts when the sleep2 monitor is done and ends, when the right_monitor is done,
# meaning it continues until the joint goal was reached.
giskard_wrapper.motion_goals.add_joint_position(goal_state=right_arm_goal,
                                                name='right pose',
                                                start_condition=sleep2,
                                                end_condition=right_monitor)
# You can use add_motion_goal to add any motion goal implemented in giskardpy.goals.
# All remaining parameters are forwarded to the __init__ function of that class.
giskard_wrapper.motion_goals.add_motion_goal(motion_goal_class=JointPositionList.__name__,
                                             goal_state=left_arm_goal,
                                             name='left pose',
                                             end_condition=left_monitor)

# %% Now let's define a goal for the base, 2m in front of it.
# First we define a monitor which checks if that pose was reached.
base_monitor = giskard_wrapper.monitors.add_cartesian_pose(root_link='map',
                                                           tip_link='base_footprint',
                                                           goal_pose=base_goal)

# and then we define a motion goal for it.
# The hold_condition causes the motion goal to hold as long as the condition is True.
# In this case, the cart pose is halted if time % 2 == 1 and active if time % 2 == 0.
giskard_wrapper.motion_goals.add_cartesian_pose(root_link='map',
                                                tip_link='base_footprint',
                                                goal_pose=base_goal,
                                                hold_condition=f'not {alternator}',
                                                end_condition=base_monitor)

# %% Define when the motion should end.
# Usually you'd use the local minimum reached monitor for this.
# Most monitors also have a stay_true parameter (when it makes sense), with reasonable default values.
# In this case, we don't want the local minimum reached monitor to stay True, because it might get triggered during
# the sleeps and therefore set it to False.
local_min = giskard_wrapper.monitors.add_local_minimum_reached(stay_true=False)

# Giskard will only end the motion generation and return Success, if an end monitor becomes True.
# We do this by defining one that gets triggered, when a local minimum was reached, sleep2 is done and the motion goals
# were reached.
giskard_wrapper.monitors.add_end_motion(start_condition=' and '.join([local_min,
                                                                      sleep2,
                                                                      right_monitor,
                                                                      left_monitor,
                                                                      base_monitor]))
# It's good to also add a cancel condition in case something went wrong and the end motion monitor is unable to become
# True. Currently, the only predefined specialized cancel monitor is max trajectory length.
# Alternative you can use monitor.add_cancel_motion similar to end_motion.
giskard_wrapper.monitors.add_max_trajectory_length(120)
# Lastly we allow all collisions
giskard_wrapper.motion_goals.allow_all_collisions()
# And execute the goal.
rospy.loginfo('Sending first goal.')
giskard_wrapper.execute()
rospy.loginfo('First goal finished.')

# %% manipulate world
box_name = 'muh'
box_pose = PoseStamped()
box_pose.header.frame_id = 'r_gripper_tool_frame'
box_pose.pose.orientation.w = 1
rospy.loginfo('Add box.')
giskard_wrapper.world.add_box(name=box_name,
                              size=(0.2, 0.1, 0.1),
                              pose=box_pose,
                              parent_link='map')
rospy.loginfo('Clear world.')
giskard_wrapper.world.clear()

rospy.loginfo('Add box again.')
giskard_wrapper.world.add_box(name=box_name,
                              size=(0.2, 0.1, 0.1),
                              pose=box_pose,
                              parent_link='map')

rospy.loginfo('Attach box at gripper.')
giskard_wrapper.world.update_parent_link_of_group(name=box_name,
                                                  parent_link='r_gripper_tool_frame')

rospy.loginfo('Delete box.')
giskard_wrapper.world.remove_group(name=box_name)

rospy.loginfo('Add a new box directly at gripper.')
giskard_wrapper.world.add_box(name=box_name,
                              size=(0.2, 0.1, 0.1),
                              pose=box_pose,
                              parent_link='r_gripper_tool_frame')

# All objects added to the world can be used as root or tip links in most motion goals or monitors.
# In this case we use the box name to set a goal for the box attached to the robot.
box_goal = PoseStamped()
box_goal.header.frame_id = box_name
box_goal.pose.position.x = 0.5
box_goal.pose.orientation.w = 1
giskard_wrapper.motion_goals.add_cartesian_pose(goal_pose=box_goal,
                                                tip_link=box_name,
                                                root_link='map')

# If you don't want to create complicated monitor/motion goal chains, the default ending conditions might be sufficient.
giskard_wrapper.add_default_end_motion_conditions()
rospy.loginfo('Send cartesian goal for box.')
giskard_wrapper.execute()
rospy.loginfo('Done.')

## Todos:
- Add other robots